In [ ]:
!pip install tweepy==4.10.0 --q
!pip install wget --q

In [1]:
import tweepy
import json
import numpy as np
import pandas as pd
from PIL import Image
import wget

In [2]:
print('TweePy version is:', tweepy.__version__)

TweePy version is: 4.10.0


In [3]:
def authenticate(creds_path):
    '''
    read the credentials and authenticate to Twitter 
    :param creds_path: str, containing path to credentials file
    :return: api, the authenticated API object 
    '''
    with open(creds_path, mode='r') as f: creds = json.load(f)
    C_KEY, C_SECRET = creds['api_key'], creds['api_key_secret']
    A_TOKEN, A_TOKEN_SECRET = creds['access_token'], creds['access_token_secret']
    auth = tweepy.OAuthHandler(consumer_key=C_KEY, consumer_secret=C_SECRET)
    auth.set_access_token(key=A_TOKEN, secret=A_TOKEN_SECRET)
    api = tweepy.API(auth=auth, wait_on_rate_limit=True)
    return api

In [4]:
USR_NAME = 'MariusCrypt0'

api = authenticate(creds_path='/content/twitter_creds.json')
timeline = api.user_timeline(screen_name=USR_NAME, count=50, exclude_replies=True, include_rts=True,
                             tweet_mode='extended')

In [5]:
timeline_df = pd.DataFrame(data=[[len(tweet.full_text), tweet.full_text, tweet.created_at, tweet.user.screen_name] for tweet in timeline], columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
timeline_df.head(10)

,Tweet_Length,Tweet_Text,Tweet_Date,UserName
0,261,1/6\n\n$CAI has done the unthinkable.\n\nThey'...,2023-03-07 22:07:11+00:00,MariusCrypt0
1,140,RT @teh_bag: Missed the first NFT WL event?\n\...,2023-03-07 21:10:12+00:00,MariusCrypt0
2,139,RT @CryptoKaduna: 1/ The #DeFi space is ever-e...,2023-03-07 19:34:37+00:00,MariusCrypt0
3,302,$BAG is building or has already built the bigg...,2023-03-07 16:07:13+00:00,MariusCrypt0
4,301,Everybody that wanted to sell $BAG and take pr...,2023-03-07 15:36:06+00:00,MariusCrypt0
5,74,RT @rovercrc: #Bitcoin Ascending Broadening We...,2023-03-07 10:42:31+00:00,MariusCrypt0


In [14]:
def extract_status_time(tweet):
    return tweet.created_at.strftime('%a, %d/%B/%Y, %H:%M %Z')


def extract_tags_and_symbols(tweet, is_retweet):
    if is_retweet: 
        tags = tweet.retweeted_status.entities['hashtags']
        symbols = tweet.retweeted_status.entities['symbols']
    else:
        tags = tweet.entities['hashtags']
        symbols = tweet.entities['symbols']
    tags = ['#' + tag['text'] for tag in tags]
    symbols = ['$' + symbol['text'] for symbol in symbols]
    return tags, symbols


def download_media(tweet, is_retweet):
    if is_retweet:
        media_files = tweet.retweeted_status.entities.get('media', [])
    else:
        media_files = tweet.entities.get('media', [])

    for idx, media_file in enumerate(media_files):
        media_url = media_file['media_url']
        media_name = tweet.id_str + '_' + str(idx) + media_url[media_url.rfind('.'):]
        img = wget.download(url=media_url, out=media_name)
        print(f'downloded image no. {idx+1} successfully!')

    if not(len(media_files)): print('no downloadable images found!')
    return None


def print_author(tweet, is_retweet):
    if is_retweet:
        original_author_name = tweet.entities.get('user_mentions')[0].get('name')
        original_author_handle = tweet.entities.get('user_mentions')[0].get('screen_name')
        print('RT')
        print(original_author_name + ' @' + original_author_handle)
    return None


def extract_full_text(tweet, is_retweet):
    return tweet.retweeted_status.full_text if is_retweet else tweet.full_text

In [17]:
for status in timeline:
    print('--------------------------------------')
    is_retweet = status.full_text.startswith('RT')
    status_time = extract_status_time(status)
    print('tweet posted at: ', status_time)
    tags, symbols = extract_tags_and_symbols(status, is_retweet=is_retweet)
    print('tweet has these tags:', tags)
    print('tweet has these symbols:', symbols)
    print('main author: ', print_author(status, is_retweet=is_retweet))
    text = extract_full_text(status, is_retweet=is_retweet)
    print('full tweet text: \n', text)
    print('--------------------------------------')

--------------------------------------
tweet posted at:  Tue, 07/March/2023, 22:07 UTC
tweet has these tags: []
tweet has these symbols: ['$CAI', '$CAI']
main author:  None
full tweet text: 
 1/6

$CAI has done the unthinkable.

They've made Midjourney for free. Anybody can now use Midjourney on Telegram for free and if they stake $300 worth of $CAI tokens, they get faster generation times and 4 image at once generation.

But they've done even more.
--------------------------------------
--------------------------------------
tweet posted at:  Tue, 07/March/2023, 21:10 UTC
tweet has these tags: ['#tehBag']
tweet has these symbols: []
RT
teh Bag - $BAG @teh_bag
main author:  None
full tweet text: 
 Missed the first NFT WL event?

A new #tehBag burning event will start tomorrow!

Don't miss teh chance to grab a Whitelist spot for the NFT minting 📈

More details tomorrow,  stay tuned ⏳ https://t.co/56XJdQUgYA
--------------------------------------
--------------------------------------
tw